In [2]:

import warnings

warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import os

import talib

import gymnasium as gym


import collections
import argparse

import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count

import seaborn as sns
import pandas as pd

In [3]:
zd = pd.read_csv('Fileterd_Data').copy()

In [2]:
# df = pd.read_csv('auth.txt')

In [3]:

# chunk_size = 100000  # Adjust the chunk size according to your available memory
# chunks = []
# count = 0
# for chunk in pd.read_csv('auth.txt', delimiter='\t', chunksize=chunk_size):
#     # Process each chunk or store it in a list for later processing
#     chunks.append(chunk)
#     count += 1
#     if count == 5:
#         break

# # Concatenate the chunks into a single DataFrame if needed
# df = pd.concat(chunks)

In [2]:

chunk_size = 100000  # Adjust the chunk size according to your available memory
chunks = []
count = 0
for chunk in pd.read_csv('auth.txt', chunksize=chunk_size):
    # Process each chunk or store it in a list for later processing
    chunks.append(chunk)
    count += 1
    if count == 2000:
        break

# Concatenate the chunks into a single DataFrame if needed
df = pd.concat(chunks)

df = df.rename({'1':'time', 'ANONYMOUS LOGON@C586':'source', 'ANONYMOUS LOGON@C586.1':'dest', 'C1250':'source_pc', 'C586':'dest_pc', 'NTLM':'auth_type', 'Network':'logon_type', 'LogOn':'auth_orient'},axis=1)
df = df[1:]
df = df.set_index('time')

del chunk, chunks

In [3]:
df

,source,dest,source_pc,dest_pc,auth_type,logon_type,auth_orient,Success
time,,,,,,,,
1,C101$@DOM1,C101$@DOM1,C988,C988,?,Network,LogOff,Success
1,C1020$@DOM1,SYSTEM@C1020,C1020,C1020,Negotiate,Service,LogOn,Success
1,C1021$@DOM1,C1021$@DOM1,C1021,C625,Kerberos,Network,LogOn,Success
1,C1035$@DOM1,C1035$@DOM1,C1035,C586,Kerberos,Network,LogOn,Success
1,C1035$@DOM1,C1035$@DOM1,C586,C586,?,Network,LogOff,Success
...,...,...,...,...,...,...,...,...
1044281,C586$@DOM1,C586$@DOM1,C586,C586,?,Network,LogOff,Success
1044281,C586$@DOM1,C586$@DOM1,C586,C586,Kerberos,Network,LogOn,Success
1044281,C599$@DOM1,C599$@DOM1,C1619,C523,Kerberos,Network,LogOn,Success


In [14]:
df.index

Int64Index([      1,       1,       1,       1,       1,       1,       1,
                  1,       1,       1,
            ...
            1044281, 1044281, 1044281, 1044281, 1044281, 1044281, 1044281,
            1044281, 1044281, 1044281],
           dtype='int64', name='time', length=199999999)

In [15]:
df.loc[150000:]

,source,dest,source_pc,dest_pc,auth_type,logon_type,auth_orient,Success
time,,,,,,,,
150000,ANONYMOUS LOGON@C457,ANONYMOUS LOGON@C457,C8017,C457,NTLM,Network,LogOn,Success
150000,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C5830,C586,NTLM,Network,LogOn,Success
150000,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C586,C586,?,Network,LogOff,Success
150000,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C5995,C586,NTLM,Network,LogOn,Success
150000,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C6480,C586,NTLM,Network,LogOn,Success
...,...,...,...,...,...,...,...,...
1044281,C586$@DOM1,C586$@DOM1,C586,C586,?,Network,LogOff,Success
1044281,C586$@DOM1,C586$@DOM1,C586,C586,Kerberos,Network,LogOn,Success
1044281,C599$@DOM1,C599$@DOM1,C1619,C523,Kerberos,Network,LogOn,Success


In [ ]:
df.loc

In [6]:
del df

In [4]:
red = pd.read_csv('redteam.txt')
red = red.rename({'150885':'time', 'U620@DOM1':'user_domain', 'C17693':'source_pc', 'C1003':'dest_pc'},axis=1)
red = red.set_index('time')
red

,user_domain,source_pc,dest_pc
time,,,
151036,U748@DOM1,C17693,C305
151648,U748@DOM1,C17693,C728
151993,U6115@DOM1,C17693,C1173
153792,U636@DOM1,C17693,C294
155219,U748@DOM1,C17693,C5693
...,...,...,...
2552687,U655@DOM1,C17693,C828
2552687,U655@DOM1,C17693,C828
2552687,U655@DOM1,C17693,C828


In [5]:
red

,user_domain,source_pc,dest_pc
time,,,
151036,U748@DOM1,C17693,C305
151648,U748@DOM1,C17693,C728
151993,U6115@DOM1,C17693,C1173
153792,U636@DOM1,C17693,C294
155219,U748@DOM1,C17693,C5693
...,...,...,...
2552687,U655@DOM1,C17693,C828
2552687,U655@DOM1,C17693,C828
2552687,U655@DOM1,C17693,C828


In [25]:
target = red.head(300)

In [24]:
red.head(330)

,user_domain,source_pc,dest_pc
time,,,
151036,U748@DOM1,C17693,C305
151648,U748@DOM1,C17693,C728
151993,U6115@DOM1,C17693,C1173
153792,U636@DOM1,C17693,C294
155219,U748@DOM1,C17693,C5693
...,...,...,...
830548,U1653@DOM1,C22409,C754
830548,U1653@DOM1,C22409,C754
830550,U1653@DOM1,C22409,C754


In [101]:
red[50:250]

,user_domain,source_pc,dest_pc
time,,,
725589,U1653@DOM1,C17693,C2669
725983,U293@DOM1,C17693,C3153
726008,U8946@DOM1,C17693,C1610
726057,U293@DOM1,C17693,C3153
726137,U10379@C3521,C17693,C3521
...,...,...,...
767289,U7394@DOM1,C17693,C492
767300,U7394@DOM1,C17693,C1797
767320,U7394@DOM1,C17693,C1797


In [5]:
target = red[50:250]

In [79]:
red.head(300)

,user_domain,source_pc,dest_pc
time,,,
151036,U748@DOM1,C17693,C305
151648,U748@DOM1,C17693,C728
151993,U6115@DOM1,C17693,C1173
153792,U636@DOM1,C17693,C294
155219,U748@DOM1,C17693,C5693
...,...,...,...
770950,U66@DOM1,C17693,C742
771133,U7375@DOM1,C17693,C1626
771134,U4448@DOM1,C17693,C1555


In [102]:
df.loc[705000:775000]

,source,dest,source_pc,dest_pc,auth_type,logon_type,auth_orient,Success
time,,,,,,,,
705000,ANONYMOUS LOGON@C457,ANONYMOUS LOGON@C457,C457,C457,?,Network,LogOff,Success
705000,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C10421,C586,NTLM,Network,LogOn,Success
705000,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C11677,C586,NTLM,Network,LogOn,Success
705000,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C14326,C586,NTLM,Network,LogOn,Success
705000,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C178,C586,NTLM,Network,LogOn,Success
...,...,...,...,...,...,...,...,...
775000,U8515@DOM1,U8515@DOM1,C18289,C528,Kerberos,Network,LogOn,Success
775000,U8515@DOM1,U8515@DOM1,C528,C528,?,Network,LogOff,Success
775000,U8695@DOM1,U8695@DOM1,C1015,C1015,?,Network,LogOff,Success


In [6]:
zd

,Unnamed: 0,Time Stamp,Source User @ Domain,Destination User @ Domain,Source Computer,Source Destination,Authentication Type,Logon Type,Auth Orientation,Success/Failure
0,137649533,715589,ANONYMOUS LOGON@C457,ANONYMOUS LOGON@C457,C457,C457,?,Network,LogOff,Success
1,137649534,715589,ANONYMOUS LOGON@C467,ANONYMOUS LOGON@C467,C467,C467,?,Network,LogOff,Success
2,137649535,715589,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C586,C586,?,Network,LogOff,Success
3,137649536,715589,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C5995,C586,NTLM,Network,LogOn,Success
4,137649537,715589,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C6931,C586,NTLM,Network,LogOn,Success
...,...,...,...,...,...,...,...,...,...,...
14863570,152513103,777813,U8857@DOM1,U8857@DOM1,C528,C528,?,Network,LogOff,Success
14863571,152513104,777813,U953@DOM1,U953@DOM1,C803,C1065,Kerberos,Network,LogOn,Success
14863572,152513105,777813,U9861@DOM1,U9861@DOM1,C457,C457,?,Network,LogOff,Success
14863573,152513106,777813,U996@DOM1,U996@DOM1,C10,C10,?,Network,LogOff,Success


In [105]:
new = df.loc[target.index[0]:target.index[-1]]
new

,source,dest,source_pc,dest_pc,auth_type,logon_type,auth_orient,Success
time,,,,,,,,
725589,ANONYMOUS LOGON@C467,ANONYMOUS LOGON@C467,C467,C467,?,Network,LogOff,Success
725589,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C3745,C586,NTLM,Network,LogOn,Success
725589,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C6363,C586,NTLM,Network,LogOn,Success
725589,ANONYMOUS LOGON@C612,ANONYMOUS LOGON@C612,C8113,C612,NTLM,Network,LogOn,Success
725589,ANONYMOUS LOGON@C625,ANONYMOUS LOGON@C625,C625,C625,?,Network,LogOff,Success
...,...,...,...,...,...,...,...,...
767813,U9556@DOM1,U9556@DOM1,C457,C457,?,Network,LogOff,Success
767813,U9556@DOM1,U9556@DOM1,C586,C586,?,Network,LogOff,Success
767813,U959@DOM1,U959@DOM1,C1065,C1065,?,Network,LogOff,Success


In [113]:
new = df.loc[(target.index[0] - 10000) : (target.index[-1] + 10000)]

In [114]:
new

,source,dest,source_pc,dest_pc,auth_type,logon_type,auth_orient,Success
time,,,,,,,,
715589,ANONYMOUS LOGON@C457,ANONYMOUS LOGON@C457,C457,C457,?,Network,LogOff,Success
715589,ANONYMOUS LOGON@C467,ANONYMOUS LOGON@C467,C467,C467,?,Network,LogOff,Success
715589,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C586,C586,?,Network,LogOff,Success
715589,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C5995,C586,NTLM,Network,LogOn,Success
715589,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C6931,C586,NTLM,Network,LogOn,Success
...,...,...,...,...,...,...,...,...
777813,U8857@DOM1,U8857@DOM1,C528,C528,?,Network,LogOff,Success
777813,U953@DOM1,U953@DOM1,C803,C1065,Kerberos,Network,LogOn,Success
777813,U9861@DOM1,U9861@DOM1,C457,C457,?,Network,LogOff,Success


In [112]:
del new

In [9]:
df

,source,dest,source_pc,dest_pc,auth_type,logon_type,auth_orient,Success
time,,,,,,,,
1,C101$@DOM1,C101$@DOM1,C988,C988,?,Network,LogOff,Success
1,C1020$@DOM1,SYSTEM@C1020,C1020,C1020,Negotiate,Service,LogOn,Success
1,C1021$@DOM1,C1021$@DOM1,C1021,C625,Kerberos,Network,LogOn,Success
1,C1035$@DOM1,C1035$@DOM1,C1035,C586,Kerberos,Network,LogOn,Success
1,C1035$@DOM1,C1035$@DOM1,C586,C586,?,Network,LogOff,Success
...,...,...,...,...,...,...,...,...
298697,U94@DOM1,U2@DOM1,C1679,C1679,?,?,AuthMap,Success
298697,U94@DOM1,U94@DOM1,C1679,C1679,?,?,TGS,Success
298697,U94@DOM1,U94@DOM1,C1679,C1679,?,?,TGT,Success


In [6]:
target

,user_domain,source_pc,dest_pc
time,,,
725589,U1653@DOM1,C17693,C2669
725983,U293@DOM1,C17693,C3153
726008,U8946@DOM1,C17693,C1610
726057,U293@DOM1,C17693,C3153
726137,U10379@C3521,C17693,C3521
...,...,...,...
767289,U7394@DOM1,C17693,C492
767300,U7394@DOM1,C17693,C1797
767320,U7394@DOM1,C17693,C1797


In [8]:
zd

,Unnamed: 0,Time Stamp,Source User @ Domain,Destination User @ Domain,Source Computer,Source Destination,Authentication Type,Logon Type,Auth Orientation,Success/Failure
0,137649533,715589,ANONYMOUS LOGON@C457,ANONYMOUS LOGON@C457,C457,C457,?,Network,LogOff,Success
1,137649534,715589,ANONYMOUS LOGON@C467,ANONYMOUS LOGON@C467,C467,C467,?,Network,LogOff,Success
2,137649535,715589,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C586,C586,?,Network,LogOff,Success
3,137649536,715589,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C5995,C586,NTLM,Network,LogOn,Success
4,137649537,715589,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C6931,C586,NTLM,Network,LogOn,Success
...,...,...,...,...,...,...,...,...,...,...
14863570,152513103,777813,U8857@DOM1,U8857@DOM1,C528,C528,?,Network,LogOff,Success
14863571,152513104,777813,U953@DOM1,U953@DOM1,C803,C1065,Kerberos,Network,LogOn,Success
14863572,152513105,777813,U9861@DOM1,U9861@DOM1,C457,C457,?,Network,LogOff,Success
14863573,152513106,777813,U996@DOM1,U996@DOM1,C10,C10,?,Network,LogOff,Success


In [9]:
target.values[0]

array(['U1653@DOM1', 'C17693', 'C2669'], dtype=object)

In [7]:
target_array = []
for i in target.values:
    result = " ".join(i)
    target_array.append(result)

In [8]:
target_array

['U1653@DOM1 C17693 C2669',
 'U293@DOM1 C17693 C3153',
 'U8946@DOM1 C17693 C1610',
 'U293@DOM1 C17693 C3153',
 'U10379@C3521 C17693 C3521',
 'U293@DOM1 C17693 C3153',
 'U8946@DOM1 C17693 C1124',
 'U293@DOM1 C17693 C3153',
 'U8946@DOM1 C17693 C3199',
 'U293@DOM1 C17693 C3153',
 'U8946@DOM1 C17693 C11727',
 'U8601@DOM1 C17693 C754',
 'U212@DOM1 C17693 C126',
 'U212@DOM1 C17693 C126',
 'U212@DOM1 C17693 C126',
 'U8946@DOM1 C17693 C9945',
 'U8946@DOM1 C17693 C9945',
 'U8946@DOM1 C17693 C9945',
 'U4978@DOM1 C17693 C1448',
 'U8946@DOM1 C17693 C9945',
 'U3905@DOM1 C17693 C9945',
 'U995@DOM1 C17693 C1014',
 'U8601@DOM1 C17693 C20819',
 'U8601@DOM1 C17693 C20819',
 'U8601@DOM1 C17693 C18626',
 'U8601@DOM1 C17693 C18626',
 'U288@DOM1 C17693 C1222',
 'U2837@DOM1 C17693 C89',
 'U2837@DOM1 C17693 C1484',
 'U2837@DOM1 C17693 C8490',
 'U349@DOM1 C17693 C3601',
 'U2837@DOM1 C17693 C1784',
 'U2837@DOM1 C17693 C1784',
 'U2837@DOM1 C17693 C1784',
 'U2837@DOM1 C17693 C2519',
 'U8946@DOM1 C17693 C11194',
 

In [116]:
del target_array

In [40]:
df

,source,dest,source_pc,dest_pc,auth_type,logon_type,auth_orient,Success
time,,,,,,,,
1,C101$@DOM1,C101$@DOM1,C988,C988,?,Network,LogOff,Success
1,C1020$@DOM1,SYSTEM@C1020,C1020,C1020,Negotiate,Service,LogOn,Success
1,C1021$@DOM1,C1021$@DOM1,C1021,C625,Kerberos,Network,LogOn,Success
1,C1035$@DOM1,C1035$@DOM1,C1035,C586,Kerberos,Network,LogOn,Success
1,C1035$@DOM1,C1035$@DOM1,C586,C586,?,Network,LogOff,Success
...,...,...,...,...,...,...,...,...
1044281,C586$@DOM1,C586$@DOM1,C586,C586,?,Network,LogOff,Success
1044281,C586$@DOM1,C586$@DOM1,C586,C586,Kerberos,Network,LogOn,Success
1044281,C599$@DOM1,C599$@DOM1,C1619,C523,Kerberos,Network,LogOn,Success


In [65]:
del new

In [9]:
from numba import jit

In [10]:
@jit
def get_loop(input_array):
    main_array = []
    for i in input_array:
        result = " ".join(i)
        main_array.append(result)
    return main_array


In [11]:

new = zd.copy()

In [12]:
new.columns

Index(['Unnamed: 0', 'Time Stamp', 'Source User @ Domain',
       'Destination User @ Domain', 'Source Computer', 'Source Destination',
       'Authentication Type', 'Logon Type', 'Auth Orientation',
       'Success/Failure'],
      dtype='object')

In [13]:
df = df.rename({'1':'time', 'ANONYMOUS LOGON@C586':'source', 'ANONYMOUS LOGON@C586.1':'dest', 'C1250':'source_pc', 'C586':'dest_pc', 'NTLM':'auth_type', 'Network':'logon_type', 'LogOn':'auth_orient'},axis=1)

NameError: name 'df' is not defined

In [14]:
new = new.rename({
    'Source User @ Domain': 'Source_Dome', 'Destination User @ Domain': 'Dest_Dome', 'Source Computer':'Source_PC', 'Source Destination':'Destination_PC', 
    'Authentication Type':'auth_type', 'Logon Type':'logon_type', 'Success/Failure':'result',
},axis=1)

In [15]:
new

,Unnamed: 0,Time Stamp,Source_Dome,Dest_Dome,Source_PC,Destination_PC,auth_type,logon_type,Auth Orientation,result
0,137649533,715589,ANONYMOUS LOGON@C457,ANONYMOUS LOGON@C457,C457,C457,?,Network,LogOff,Success
1,137649534,715589,ANONYMOUS LOGON@C467,ANONYMOUS LOGON@C467,C467,C467,?,Network,LogOff,Success
2,137649535,715589,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C586,C586,?,Network,LogOff,Success
3,137649536,715589,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C5995,C586,NTLM,Network,LogOn,Success
4,137649537,715589,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C6931,C586,NTLM,Network,LogOn,Success
...,...,...,...,...,...,...,...,...,...,...
14863570,152513103,777813,U8857@DOM1,U8857@DOM1,C528,C528,?,Network,LogOff,Success
14863571,152513104,777813,U953@DOM1,U953@DOM1,C803,C1065,Kerberos,Network,LogOn,Success
14863572,152513105,777813,U9861@DOM1,U9861@DOM1,C457,C457,?,Network,LogOff,Success
14863573,152513106,777813,U996@DOM1,U996@DOM1,C10,C10,?,Network,LogOff,Success


In [22]:
new[['Source_Dome', 'Source_PC', 'Destination_PC']].values[0]

array(['ANONYMOUS LOGON@C457', 'C457', 'C457'], dtype=object)

In [16]:
main = get_loop(new[['Source_Dome', 'Source_PC', 'Destination_PC']].values)

In [23]:
main = get_loop(new[['source', 'source_pc', 'dest_pc']].values)

KeyError: "None of [Index(['source', 'source_pc', 'dest_pc'], dtype='object')] are in the [columns]"

In [17]:
new['test'] = main

In [18]:
new['target'] = new['test'].apply(lambda x: 1 if x in target_array else 0)

In [19]:
new

,Unnamed: 0,Time Stamp,Source_Dome,Dest_Dome,Source_PC,Destination_PC,auth_type,logon_type,Auth Orientation,result,test,target
0,137649533,715589,ANONYMOUS LOGON@C457,ANONYMOUS LOGON@C457,C457,C457,?,Network,LogOff,Success,ANONYMOUS LOGON@C457 C457 C457,0
1,137649534,715589,ANONYMOUS LOGON@C467,ANONYMOUS LOGON@C467,C467,C467,?,Network,LogOff,Success,ANONYMOUS LOGON@C467 C467 C467,0
2,137649535,715589,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C586,C586,?,Network,LogOff,Success,ANONYMOUS LOGON@C586 C586 C586,0
3,137649536,715589,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C5995,C586,NTLM,Network,LogOn,Success,ANONYMOUS LOGON@C586 C5995 C586,0
4,137649537,715589,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C6931,C586,NTLM,Network,LogOn,Success,ANONYMOUS LOGON@C586 C6931 C586,0
...,...,...,...,...,...,...,...,...,...,...,...,...
14863570,152513103,777813,U8857@DOM1,U8857@DOM1,C528,C528,?,Network,LogOff,Success,U8857@DOM1 C528 C528,0
14863571,152513104,777813,U953@DOM1,U953@DOM1,C803,C1065,Kerberos,Network,LogOn,Success,U953@DOM1 C803 C1065,0
14863572,152513105,777813,U9861@DOM1,U9861@DOM1,C457,C457,?,Network,LogOff,Success,U9861@DOM1 C457 C457,0
14863573,152513106,777813,U996@DOM1,U996@DOM1,C10,C10,?,Network,LogOff,Success,U996@DOM1 C10 C10,0


In [20]:
new = new.set_index('Time Stamp')
new

,Unnamed: 0,Source_Dome,Dest_Dome,Source_PC,Destination_PC,auth_type,logon_type,Auth Orientation,result,test,target
Time Stamp,,,,,,,,,,,
715589,137649533,ANONYMOUS LOGON@C457,ANONYMOUS LOGON@C457,C457,C457,?,Network,LogOff,Success,ANONYMOUS LOGON@C457 C457 C457,0
715589,137649534,ANONYMOUS LOGON@C467,ANONYMOUS LOGON@C467,C467,C467,?,Network,LogOff,Success,ANONYMOUS LOGON@C467 C467 C467,0
715589,137649535,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C586,C586,?,Network,LogOff,Success,ANONYMOUS LOGON@C586 C586 C586,0
715589,137649536,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C5995,C586,NTLM,Network,LogOn,Success,ANONYMOUS LOGON@C586 C5995 C586,0
715589,137649537,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C6931,C586,NTLM,Network,LogOn,Success,ANONYMOUS LOGON@C586 C6931 C586,0
...,...,...,...,...,...,...,...,...,...,...,...
777813,152513103,U8857@DOM1,U8857@DOM1,C528,C528,?,Network,LogOff,Success,U8857@DOM1 C528 C528,0
777813,152513104,U953@DOM1,U953@DOM1,C803,C1065,Kerberos,Network,LogOn,Success,U953@DOM1 C803 C1065,0
777813,152513105,U9861@DOM1,U9861@DOM1,C457,C457,?,Network,LogOff,Success,U9861@DOM1 C457 C457,0


In [22]:
new.to_csv('Full_Data.csv')

In [23]:
temp = pd.read_csv('Full_Data.csv')

In [24]:
temp.columns

Index(['Time Stamp', 'Unnamed: 0', 'Source_Dome', 'Dest_Dome', 'Source_PC',
       'Destination_PC', 'auth_type', 'logon_type', 'Auth Orientation',
       'result', 'test', 'target'],
      dtype='object')

In [25]:
temp = temp.drop(['Unnamed: 0','test'],axis=1)
temp

,Time Stamp,Source_Dome,Dest_Dome,Source_PC,Destination_PC,auth_type,logon_type,Auth Orientation,result,target
0,715589,ANONYMOUS LOGON@C457,ANONYMOUS LOGON@C457,C457,C457,?,Network,LogOff,Success,0
1,715589,ANONYMOUS LOGON@C467,ANONYMOUS LOGON@C467,C467,C467,?,Network,LogOff,Success,0
2,715589,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C586,C586,?,Network,LogOff,Success,0
3,715589,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C5995,C586,NTLM,Network,LogOn,Success,0
4,715589,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C6931,C586,NTLM,Network,LogOn,Success,0
...,...,...,...,...,...,...,...,...,...,...
14863570,777813,U8857@DOM1,U8857@DOM1,C528,C528,?,Network,LogOff,Success,0
14863571,777813,U953@DOM1,U953@DOM1,C803,C1065,Kerberos,Network,LogOn,Success,0
14863572,777813,U9861@DOM1,U9861@DOM1,C457,C457,?,Network,LogOff,Success,0
14863573,777813,U996@DOM1,U996@DOM1,C10,C10,?,Network,LogOff,Success,0


In [26]:
temp['result'] = np.where(temp['result']=='Success', 1, 0)

In [29]:
temp = temp.set_index('Time Stamp')

In [31]:
temp['Auth Orientation'].value_counts()

LogOff          6258102
LogOn           6212477
TGS             1706660
TGT              502946
AuthMap          178085
ScreenLock         2713
ScreenUnlock       2592
Name: Auth Orientation, dtype: int64

In [33]:
from sklearn.preprocessing import OneHotEncoder

In [34]:
enc = OneHotEncoder()

In [35]:
train = temp.drop(['target'],axis=1)


In [32]:
temp

,Source_Dome,Dest_Dome,Source_PC,Destination_PC,auth_type,logon_type,Auth Orientation,result,target
Time Stamp,,,,,,,,,
715589,ANONYMOUS LOGON@C457,ANONYMOUS LOGON@C457,C457,C457,?,Network,LogOff,1,0
715589,ANONYMOUS LOGON@C467,ANONYMOUS LOGON@C467,C467,C467,?,Network,LogOff,1,0
715589,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C586,C586,?,Network,LogOff,1,0
715589,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C5995,C586,NTLM,Network,LogOn,1,0
715589,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C6931,C586,NTLM,Network,LogOn,1,0
...,...,...,...,...,...,...,...,...,...
777813,U8857@DOM1,U8857@DOM1,C528,C528,?,Network,LogOff,1,0
777813,U953@DOM1,U953@DOM1,C803,C1065,Kerberos,Network,LogOn,1,0
777813,U9861@DOM1,U9861@DOM1,C457,C457,?,Network,LogOff,1,0


In [27]:
temp['result']

0           1
1           1
2           1
3           1
4           1
           ..
14863570    1
14863571    1
14863572    1
14863573    1
14863574    1
Name: result, Length: 14863575, dtype: int32

In [21]:
new.loc[715599]

,Unnamed: 0,Source_Dome,Dest_Dome,Source_PC,Destination_PC,auth_type,logon_type,Auth Orientation,result,test,target
Time Stamp,,,,,,,,,,,
715599,137651563,ANONYMOUS LOGON@C2106,ANONYMOUS LOGON@C2106,C2106,C2106,?,Network,LogOff,Success,ANONYMOUS LOGON@C2106 C2106 C2106,0
715599,137651564,ANONYMOUS LOGON@C457,ANONYMOUS LOGON@C457,C16455,C457,NTLM,Network,LogOn,Success,ANONYMOUS LOGON@C457 C16455 C457,0
715599,137651565,ANONYMOUS LOGON@C457,ANONYMOUS LOGON@C457,C3637,C457,NTLM,Network,LogOn,Success,ANONYMOUS LOGON@C457 C3637 C457,0
715599,137651566,ANONYMOUS LOGON@C457,ANONYMOUS LOGON@C457,C457,C457,?,Network,LogOff,Success,ANONYMOUS LOGON@C457 C457 C457,0
715599,137651567,ANONYMOUS LOGON@C529,ANONYMOUS LOGON@C529,C3252,C529,NTLM,Network,LogOn,Success,ANONYMOUS LOGON@C529 C3252 C529,0
...,...,...,...,...,...,...,...,...,...,...,...
715599,137651747,U8@DOM1,U8@DOM1,C62,C36,Kerberos,Network,LogOn,Success,U8@DOM1 C62 C36,0
715599,137651748,U8@DOM1,U8@DOM1,C62,C464,Kerberos,Network,LogOn,Success,U8@DOM1 C62 C464,0
715599,137651749,U94@DOM1,U94@DOM1,C7,C7,?,Network,LogOff,Success,U94@DOM1 C7 C7,0


In [40]:
new.loc[715589 + 10]

,Unnamed: 0,Source_Dome,Dest_Dome,Source_PC,Destination_PC,auth_type,logon_type,Auth Orientation,result,test,target
Time Stamp,,,,,,,,,,,
715599,137651563,ANONYMOUS LOGON@C2106,ANONYMOUS LOGON@C2106,C2106,C2106,?,Network,LogOff,Success,ANONYMOUS LOGON@C2106 C2106 C2106,0
715599,137651564,ANONYMOUS LOGON@C457,ANONYMOUS LOGON@C457,C16455,C457,NTLM,Network,LogOn,Success,ANONYMOUS LOGON@C457 C16455 C457,0
715599,137651565,ANONYMOUS LOGON@C457,ANONYMOUS LOGON@C457,C3637,C457,NTLM,Network,LogOn,Success,ANONYMOUS LOGON@C457 C3637 C457,0
715599,137651566,ANONYMOUS LOGON@C457,ANONYMOUS LOGON@C457,C457,C457,?,Network,LogOff,Success,ANONYMOUS LOGON@C457 C457 C457,0
715599,137651567,ANONYMOUS LOGON@C529,ANONYMOUS LOGON@C529,C3252,C529,NTLM,Network,LogOn,Success,ANONYMOUS LOGON@C529 C3252 C529,0
...,...,...,...,...,...,...,...,...,...,...,...
715599,137651747,U8@DOM1,U8@DOM1,C62,C36,Kerberos,Network,LogOn,Success,U8@DOM1 C62 C36,0
715599,137651748,U8@DOM1,U8@DOM1,C62,C464,Kerberos,Network,LogOn,Success,U8@DOM1 C62 C464,0
715599,137651749,U94@DOM1,U94@DOM1,C7,C7,?,Network,LogOff,Success,U94@DOM1 C7 C7,0


In [46]:
new['Source_PC']

Time Stamp
715589     C457
715589     C467
715589     C586
715589    C5995
715589    C6931
          ...  
777813     C528
777813     C803
777813     C457
777813      C10
777813    C3173
Name: Source_PC, Length: 14863575, dtype: object

In [47]:
new[
    new['Source_PC'] == 'C17693'
]

,Unnamed: 0,Source_Dome,Dest_Dome,Source_PC,Destination_PC,auth_type,logon_type,Auth Orientation,result,test,target
Time Stamp,,,,,,,,,,,
721806,139305076,U10018@DOM1,U10018@DOM1,C17693,C10171,NTLM,Network,LogOn,Fail,U10018@DOM1 C17693 C10171,0
724055,139943539,U8168@C1654,U8168@C1654,C17693,C1654,NTLM,Network,LogOn,Success,U8168@C1654 C17693 C1654,0
724161,139973223,U8168@C4747,U8168@C4747,C17693,C4747,NTLM,Network,LogOn,Success,U8168@C4747 C17693 C4747,0
725274,140295865,U8601@DOM1,U8601@DOM1,C17693,C1893,NTLM,Network,LogOn,Success,U8601@DOM1 C17693 C1893,0
725488,140356557,U1653@DOM1,U1653@DOM1,C17693,C395,NTLM,Network,LogOn,Success,U1653@DOM1 C17693 C395,1
...,...,...,...,...,...,...,...,...,...,...,...
774214,151869657,U66@DOM1,U66@DOM1,C17693,C1966,NTLM,Network,LogOn,Success,U66@DOM1 C17693 C1966,0
774246,151875330,U66@DOM1,U66@DOM1,C17693,C19646,NTLM,Network,LogOn,Fail,U66@DOM1 C17693 C19646,0
774452,151916647,U66@DOM1,U66@DOM1,C17693,C2388,NTLM,Network,LogOn,Success,U66@DOM1 C17693 C2388,0


In [48]:
new[ 
    new['target'] == 1
]

,Unnamed: 0,Source_Dome,Dest_Dome,Source_PC,Destination_PC,auth_type,logon_type,Auth Orientation,result,test,target
Time Stamp,,,,,,,,,,,
725488,140356557,U1653@DOM1,U1653@DOM1,C17693,C395,NTLM,Network,LogOn,Success,U1653@DOM1 C17693 C395,1
725589,140384250,U1653@DOM1,U1653@DOM1,C17693,C2669,NTLM,Network,LogOn,Success,U1653@DOM1 C17693 C2669,1
725983,140495442,U293@DOM1,U293@DOM1,C17693,C3153,NTLM,Network,LogOn,Success,U293@DOM1 C17693 C3153,1
726008,140503428,U8946@DOM1,U8946@DOM1,C17693,C1610,NTLM,Network,LogOn,Success,U8946@DOM1 C17693 C1610,1
726057,140516533,U293@DOM1,U293@DOM1,C17693,C3153,NTLM,Network,LogOn,Success,U293@DOM1 C17693 C3153,1
...,...,...,...,...,...,...,...,...,...,...,...
767320,150651884,U7394@DOM1,U7394@DOM1,C17693,C1797,NTLM,Network,LogOn,Success,U7394@DOM1 C17693 C1797,1
767445,150675898,U4353@DOM1,U4353@DOM1,C17693,C801,NTLM,Network,LogOn,Success,U4353@DOM1 C17693 C801,1
767813,150743322,U66@DOM1,U66@DOM1,C17693,C2944,NTLM,Network,LogOn,Success,U66@DOM1 C17693 C2944,1


In [49]:
new[
    new['logon_type'] == 'Network'
]

,Unnamed: 0,Source_Dome,Dest_Dome,Source_PC,Destination_PC,auth_type,logon_type,Auth Orientation,result,test,target
Time Stamp,,,,,,,,,,,
715589,137649533,ANONYMOUS LOGON@C457,ANONYMOUS LOGON@C457,C457,C457,?,Network,LogOff,Success,ANONYMOUS LOGON@C457 C457 C457,0
715589,137649534,ANONYMOUS LOGON@C467,ANONYMOUS LOGON@C467,C467,C467,?,Network,LogOff,Success,ANONYMOUS LOGON@C467 C467 C467,0
715589,137649535,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C586,C586,?,Network,LogOff,Success,ANONYMOUS LOGON@C586 C586 C586,0
715589,137649536,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C5995,C586,NTLM,Network,LogOn,Success,ANONYMOUS LOGON@C586 C5995 C586,0
715589,137649537,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C6931,C586,NTLM,Network,LogOn,Success,ANONYMOUS LOGON@C586 C6931 C586,0
...,...,...,...,...,...,...,...,...,...,...,...
777813,152513103,U8857@DOM1,U8857@DOM1,C528,C528,?,Network,LogOff,Success,U8857@DOM1 C528 C528,0
777813,152513104,U953@DOM1,U953@DOM1,C803,C1065,Kerberos,Network,LogOn,Success,U953@DOM1 C803 C1065,0
777813,152513105,U9861@DOM1,U9861@DOM1,C457,C457,?,Network,LogOff,Success,U9861@DOM1 C457 C457,0


In [45]:
new[ 
    new['target'] == 1
]['Source_PC'].value_counts()

C17693    205
Name: Source_PC, dtype: int64

In [53]:
new[ 
    new['target'] == 1
]['Destination_PC'].value_counts()

C3153    6
C1448    6
C395     5
C9945    5
C1784    5
        ..
C1477    1
C90      1
C2578    1
C3601    1
C2944    1
Name: Destination_PC, Length: 127, dtype: int64

In [54]:
new[ 
    new['target'] == 0
]['Destination_PC'].value_counts()

C586      1832959
C612      1194362
C529      1160703
C457      1100756
C625      1057538
           ...   
C21760          1
C21330          1
C10193          1
C10536          1
C5832           1
Name: Destination_PC, Length: 11606, dtype: int64

In [51]:
new[ 
    new['target'] == 1
]['auth_type'].value_counts()

NTLM    205
Name: auth_type, dtype: int64

In [52]:
new[ 
    new['target'] == 0
]['auth_type'].value_counts()

?                                         8652630
Kerberos                                  5088733
NTLM                                       773921
Negotiate                                  325672
MICROSOFT_AUTHENTICATION_PACKAGE_V1_0       19778
MICROSOFT_AUTHENTICATION_PACKAGE_V1_          730
MICROSOFT_AUTHENTICATION_PACKAG               583
MICROSOFT_AUTHENTICATION_PACKAGE_V1           402
MICROSOFT_AUTHENTICATION_PACKAGE              338
MICROSOFT_AUTHENTICATION_PACKAGE_             270
MICROSOFT_AUTHENTICATION_PACKA                104
MICROSOFT_AUTHENTICATION_PACKAGE_V             91
NETWARE_AUTHENTICATION_PACKAGE_V1_0            33
MICROSOFT_AUTHENTICATION_PACK                  30
MICROSOFT_AUTHENTICA                           16
MICROSOFT_AUTHENTICATION_PAC                   12
MICROSOFT_AUTHENTICATION_PA                    11
Wave                                            5
CygwinLsa                                       5
MICROSOFT_AUTHENTICATI                          3


In [50]:
new[ 
    new['target'] == 1
]['Dest_Dome'].value_counts()

U293@DOM1       30
U2837@DOM1      16
U9947@DOM1      15
U5087@DOM1      15
U3635@DOM1      13
U8946@DOM1      10
U1480@DOM1      10
U162@DOM1        8
U737@DOM1        8
U342@DOM1        7
U1450@DOM1       7
U1653@DOM1       7
U8601@DOM1       7
U66@DOM1         6
U2575@DOM1       5
U349@DOM1        4
U882@DOM1        3
U212@DOM1        3
U7394@DOM1       3
U1600@DOM1       3
U374@DOM1        2
U8777@C583       2
U4353@DOM1       2
U9763@DOM1       2
U642@DOM1        2
U314@DOM1        2
U6572@DOM1       1
U1164@DOM1       1
U10379@C3521     1
U4856@DOM1       1
U288@DOM1        1
U795@DOM1        1
U3905@DOM1       1
U3718@DOM1       1
U250@DOM1        1
U8777@C3388      1
U8777@C1500      1
U995@DOM1        1
U4978@DOM1       1
Name: Dest_Dome, dtype: int64

In [29]:
new.index

Int64Index([715589, 715589, 715589, 715589, 715589, 715589, 715589, 715589,
            715589, 715589,
            ...
            777813, 777813, 777813, 777813, 777813, 777813, 777813, 777813,
            777813, 777813],
           dtype='int64', name='Time Stamp', length=14863575)

In [127]:
new.to_csv('target_data.csv')

In [128]:
new[
    new['target'] == 1
]

,source,dest,source_pc,dest_pc,auth_type,logon_type,auth_orient,Success,test,target
time,,,,,,,,,,
725488,U1653@DOM1,U1653@DOM1,C17693,C395,NTLM,Network,LogOn,Success,U1653@DOM1 C17693 C395,1
725589,U1653@DOM1,U1653@DOM1,C17693,C2669,NTLM,Network,LogOn,Success,U1653@DOM1 C17693 C2669,1
725983,U293@DOM1,U293@DOM1,C17693,C3153,NTLM,Network,LogOn,Success,U293@DOM1 C17693 C3153,1
726008,U8946@DOM1,U8946@DOM1,C17693,C1610,NTLM,Network,LogOn,Success,U8946@DOM1 C17693 C1610,1
726057,U293@DOM1,U293@DOM1,C17693,C3153,NTLM,Network,LogOn,Success,U293@DOM1 C17693 C3153,1
...,...,...,...,...,...,...,...,...,...,...
767320,U7394@DOM1,U7394@DOM1,C17693,C1797,NTLM,Network,LogOn,Success,U7394@DOM1 C17693 C1797,1
767445,U4353@DOM1,U4353@DOM1,C17693,C801,NTLM,Network,LogOn,Success,U4353@DOM1 C17693 C801,1
767813,U66@DOM1,U66@DOM1,C17693,C2944,NTLM,Network,LogOn,Success,U66@DOM1 C17693 C2944,1


In [132]:
new['source']

time
715589    ANONYMOUS LOGON@C457
715589    ANONYMOUS LOGON@C467
715589    ANONYMOUS LOGON@C586
715589    ANONYMOUS LOGON@C586
715589    ANONYMOUS LOGON@C586
                  ...         
777813              U8857@DOM1
777813               U953@DOM1
777813              U9861@DOM1
777813               U996@DOM1
777813               U996@DOM1
Name: source, Length: 14863575, dtype: object

In [135]:
new['source'].values[0].split()

['ANONYMOUS', 'LOGON@C457']

In [136]:
test = new.copy()

In [137]:
test[['user', 'domain']] = df['source'].str.split('@', n=1, expand=True)

KeyboardInterrupt: 

In [129]:
from lightgbm import LGBMClassifier

In [130]:
model = LGBMClassifier(
    boosting_type='rf',
    num_leaves=440,
    max_depth=21,
    n_estimators=100,
    min_child_samples=10,
    n_jobs=2,
    random_state=5
)

In [126]:
new['target'].value_counts()

0    14863370
1         205
Name: target, dtype: int64

In [ ]:
final_array = []
for i in target_array:
    if i in 

In [78]:
target_array[0] in new['test'].values

True

In [ ]:
@jit
def get_me(input_array):
    final_array = []
    for i in input_array:
        

In [70]:
main

['ANONYMOUS LOGON@C1065 C1065 C1065',
 'ANONYMOUS LOGON@C457 C3110 C457',
 'ANONYMOUS LOGON@C457 C457 C457',
 'ANONYMOUS LOGON@C467 C467 C467',
 'ANONYMOUS LOGON@C586 C1083 C586',
 'ANONYMOUS LOGON@C586 C17192 C586',
 'ANONYMOUS LOGON@C586 C25347 C586',
 'ANONYMOUS LOGON@C586 C586 C586',
 'ANONYMOUS LOGON@C612 C612 C612',
 'C1009$@DOM1 C586 C586',
 'C101$@DOM1 C101 C467',
 'C101$@DOM1 C467 C467',
 'C10307$@DOM1 C10307 C3173',
 'C10349$@DOM1 C457 C457',
 'C1035$@DOM1 C1035 C467',
 'C1035$@DOM1 C467 C467',
 'C10632$@DOM1 C10632 C2106',
 'C1083$@DOM1 C1083 C586',
 'C10918$@DOM1 C467 C467',
 'C10944$@DOM1 C10944 C2106',
 'C1099$@DOM1 C457 C457',
 'C11103$@DOM1 C11103 C586',
 'C1114$@DOM1 C1115 C457',
 'C11149$@DOM1 C11149 C586',
 'C11149$@DOM1 C11149 C586',
 'C11149$@DOM1 C586 C586',
 'C1125$@DOM1 C2106 C2106',
 'C11292$@DOM1 C11292 C528',
 'C11344$@DOM1 C586 C586',
 'C11714$@DOM1 C586 C586',
 'C11829$@DOM1 C11829 C2106',
 'C12154$@DOM1 C12154 C3034',
 'C123$@DOM1 C527 C988',
 'C123$@DOM1 

In [61]:
main_array = []
for i in df[['source', 'source_pc', 'dest_pc']].values:
    result = " ".join(i)
    main_array.append(result)
main_array

KeyboardInterrupt: 

In [62]:
del main_array, i, result

In [38]:
target_tuple = tuple()
main_tuple = tuple()

In [42]:
values = []
for i in target.values:
    target_set = set(i)
    for main in df[['source', 'source_pc', 'dest_pc']].values:
        main_set = set(main)
        if target_set == main_set:
            values.append(1)
        else:
            values.append(0)

KeyboardInterrupt: 

In [37]:
main_tuple = tuple()


,source,dest,source_pc,dest_pc,auth_type,logon_type,auth_orient,Success
time,,,,,,,,
1,C101$@DOM1,C101$@DOM1,C988,C988,?,Network,LogOff,Success
1,C1020$@DOM1,SYSTEM@C1020,C1020,C1020,Negotiate,Service,LogOn,Success
1,C1021$@DOM1,C1021$@DOM1,C1021,C625,Kerberos,Network,LogOn,Success
1,C1035$@DOM1,C1035$@DOM1,C1035,C586,Kerberos,Network,LogOn,Success
1,C1035$@DOM1,C1035$@DOM1,C586,C586,?,Network,LogOff,Success
...,...,...,...,...,...,...,...,...
1044281,C586$@DOM1,C586$@DOM1,C586,C586,?,Network,LogOff,Success
1044281,C586$@DOM1,C586$@DOM1,C586,C586,Kerberos,Network,LogOn,Success
1044281,C599$@DOM1,C599$@DOM1,C1619,C523,Kerberos,Network,LogOn,Success


In [28]:
new = df.loc[target.index]

In [33]:
new_target = target.copy()
new_target = target.reset_index()
new_target

,time,user_domain,source_pc,dest_pc
0,151036,U748@DOM1,C17693,C305
1,151648,U748@DOM1,C17693,C728
2,151993,U6115@DOM1,C17693,C1173
3,153792,U636@DOM1,C17693,C294
4,155219,U748@DOM1,C17693,C5693
...,...,...,...,...
295,770950,U66@DOM1,C17693,C742
296,771133,U7375@DOM1,C17693,C1626
297,771134,U4448@DOM1,C17693,C1555
298,771340,U9763@DOM1,C17693,C353


In [34]:
new_target = new_target.set_index('source_pc')
new_target

,time,user_domain,dest_pc
source_pc,,,
C17693,151036,U748@DOM1,C305
C17693,151648,U748@DOM1,C728
C17693,151993,U6115@DOM1,C1173
C17693,153792,U636@DOM1,C294
C17693,155219,U748@DOM1,C5693
...,...,...,...
C17693,770950,U66@DOM1,C742
C17693,771133,U7375@DOM1,C1626
C17693,771134,U4448@DOM1,C1555


In [29]:
new

,source,dest,source_pc,dest_pc,auth_type,logon_type,auth_orient,Success
time,,,,,,,,
151036,ANONYMOUS LOGON@C1065,ANONYMOUS LOGON@C1065,C1065,C1065,?,Network,LogOff,Success
151036,ANONYMOUS LOGON@C529,ANONYMOUS LOGON@C529,C529,C529,?,Network,LogOff,Success
151036,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C2729,C586,NTLM,Network,LogOn,Success
151036,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C586,C586,?,Network,LogOff,Success
151036,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C8843,C586,NTLM,Network,LogOn,Success
...,...,...,...,...,...,...,...,...
771380,U9397@DOM1,U9397@DOM1,C19825,C528,Kerberos,Network,LogOn,Success
771380,U9397@DOM1,U9397@DOM1,C19825,C586,Kerberos,Network,LogOn,Success
771380,U9397@DOM1,U9397@DOM1,C528,C528,?,Network,LogOff,Success


In [30]:
new_df = new.copy()

In [31]:
new_df = new_df.reset_index()
new_df

,time,source,dest,source_pc,dest_pc,auth_type,logon_type,auth_orient,Success
0,151036,ANONYMOUS LOGON@C1065,ANONYMOUS LOGON@C1065,C1065,C1065,?,Network,LogOff,Success
1,151036,ANONYMOUS LOGON@C529,ANONYMOUS LOGON@C529,C529,C529,?,Network,LogOff,Success
2,151036,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C2729,C586,NTLM,Network,LogOn,Success
3,151036,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C586,C586,?,Network,LogOff,Success
4,151036,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C8843,C586,NTLM,Network,LogOn,Success
...,...,...,...,...,...,...,...,...,...
68010,771380,U9397@DOM1,U9397@DOM1,C19825,C528,Kerberos,Network,LogOn,Success
68011,771380,U9397@DOM1,U9397@DOM1,C19825,C586,Kerberos,Network,LogOn,Success
68012,771380,U9397@DOM1,U9397@DOM1,C528,C528,?,Network,LogOff,Success
68013,771380,U9576@DOM1,U9576@DOM1,C20197,C3173,Kerberos,Network,LogOn,Success


In [32]:
new_df = new_df.set_index('source_pc')
new_df

,time,source,dest,dest_pc,auth_type,logon_type,auth_orient,Success
source_pc,,,,,,,,
C1065,151036,ANONYMOUS LOGON@C1065,ANONYMOUS LOGON@C1065,C1065,?,Network,LogOff,Success
C529,151036,ANONYMOUS LOGON@C529,ANONYMOUS LOGON@C529,C529,?,Network,LogOff,Success
C2729,151036,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C586,NTLM,Network,LogOn,Success
C586,151036,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C586,?,Network,LogOff,Success
C8843,151036,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C586,NTLM,Network,LogOn,Success
...,...,...,...,...,...,...,...,...
C19825,771380,U9397@DOM1,U9397@DOM1,C528,Kerberos,Network,LogOn,Success
C19825,771380,U9397@DOM1,U9397@DOM1,C586,Kerberos,Network,LogOn,Success
C528,771380,U9397@DOM1,U9397@DOM1,C528,?,Network,LogOff,Success


In [35]:
new_df.loc[new_target.index]

,time,source,dest,dest_pc,auth_type,logon_type,auth_orient,Success
source_pc,,,,,,,,
C17693,151036,U748@DOM1,U748@DOM1,C305,NTLM,Network,LogOn,Success
C17693,151648,U748@DOM1,U748@DOM1,C728,NTLM,Network,LogOn,Success
C17693,151993,U6115@DOM1,U6115@DOM1,C1173,NTLM,Network,LogOn,Success
C17693,153792,U636@DOM1,U636@DOM1,C294,NTLM,Network,LogOn,Success
C17693,155219,U748@DOM1,U748@DOM1,C5693,NTLM,Network,LogOn,Success
...,...,...,...,...,...,...,...,...
C17693,770950,U66@DOM1,U66@DOM1,C742,NTLM,Network,LogOn,Success
C17693,771133,U7375@DOM1,U7375@DOM1,C1626,NTLM,Network,LogOn,Success
C17693,771134,U4448@DOM1,U4448@DOM1,C1555,NTLM,Network,LogOn,Success


In [36]:
target

,user_domain,source_pc,dest_pc
time,,,
151036,U748@DOM1,C17693,C305
151648,U748@DOM1,C17693,C728
151993,U6115@DOM1,C17693,C1173
153792,U636@DOM1,C17693,C294
155219,U748@DOM1,C17693,C5693
...,...,...,...
770950,U66@DOM1,C17693,C742
771133,U7375@DOM1,C17693,C1626
771134,U4448@DOM1,C17693,C1555


In [ ]:
items = []
for i in new['source_pc'].values:
    for n in target['source_pc'].values:
        if i == n:
            items.append()

In [ ]:
new

In [6]:
del red 
del df
# del chunks
# del chunk

NameError: name 'red' is not defined

In [6]:
del chunks
del chunk